In [ ]:
pip install -q --upgrade torch hugsvision torchvision

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import hugsvision
print(hugsvision.__version__)

In [ ]:
import torchvision
torchvision.__version__

In [ ]:
from hugsvision.dataio.VisionDataset import VisionDataset

# Las clases se sacan del nombre de la carpeta

train, test, id2label, label2id = VisionDataset.fromImageFolder(
    "./DB_SORTED_BY_TYPE",
    test_ratio   = 0.20,
    balanced     = False,
    augmentation = True,
)

In [ ]:
huggingface_model = 'google/vit-base-patch16-224-in21k'

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer

trainer = VisionClassifierTrainer(
    model_name   = "MWDTYPE",
    train      	 = train,
    test      	 = test,
    output_dir   = "./out/",
    max_epochs   = 10,
    batch_size   = 130,
    lr=1e-3,
    model = ViTForImageClassification.from_pretrained(
        huggingface_model,
        num_labels = len(label2id),
        label2id   = label2id,
        id2label   = id2label
    ),
    feature_extractor = ViTFeatureExtractor.from_pretrained(
    huggingface_model),
)

In [ ]:
model = ViTForImageClassification.from_pretrained(
        huggingface_model,
        num_labels = len(label2id),
        label2id   = label2id,
        id2label   = id2label,
        )

In [ ]:
ref, hyp = trainer.evaluate_f1_score()

In [ ]:
import os.path
from transformers import ViTFeatureExtractor, ViTForImageClassification
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference

path = "./out/MWDTYPE/5_2023-12-11-23-42-51/model" ## apuntar donde está el modelo
img  = "/home/robotica10/nuevasEtiquetas/merged_noaug/train/0b83678a-22-251332213153766_-22-228191850263972_-68-93012286027773_-68-90529322107572_png.rf.744cc1f8fa65a2a76948aafe591d6a4c.jpg" # imagen para hacer inferencia

classifier = VisionClassifierInference(
    feature_extractor = ViTFeatureExtractor.from_pretrained(path),
    model = ViTForImageClassification.from_pretrained(path),
)

label = classifier.predict(img_path=img)
print("Predicted class:", label)